In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
# !pip install folium
import folium
# !pip install geopy 
from geopy.geocoders import Nominatim

     |████████████████████████████████| 102kB 8.9MB/s ta 0:00:011


# Part I- Fetching and setting up of data


###  1.Scrap data from Wikipedia page into a DataFrame

In [2]:
toronto_df = pd.DataFrame()
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')
table = soup.find_all('table')[0] 
dfdummy=pd.read_html(str(table))[0]
data = {'PostalCode':  dfdummy["Postal Code"].tolist(),
        'Borough': dfdummy["Borough"].tolist(),
         'Neighbourhood':dfdummy["Neighbourhood"].tolist()
        }
df=pd.DataFrame(data, columns = ['PostalCode','Borough','Neighbourhood'])
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 2.Data Wrangling of the above dataframe

In [3]:
#Remove not assigned borough's
df = df[df['Borough'] != "Not assigned"] 
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
# for Neighborhood="Not assigned", make the value the same as Borough
df['Neighbourhood'] = np.where(df['Neighbourhood'] == "Not assigned",df['Borough'],df['Neighbourhood'] )
df.head()
df.shape

(103, 3)

# Part II - Setting latitude and longitude of data

### Load the coordinates from GeoCoders and CSV provided


In [5]:
#Loading the data
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ecb45f4c06a0469b818b7c12544ffe97 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='opqif3eHf3VhyOHUM9fD8wK7vgx55OKMsoTh1CDScwj5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_ecb45f4c06a0469b818b7c12544ffe97.get_object(Bucket='datascience-donotdelete-pr-piurdy8risiyyv',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df_data_1.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df = df.merge(df_data_1, on="PostalCode", how="left")
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Part III - Clustering

### Load coordinates of toronto

In [7]:
address = 'Toronto'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [10]:
df = df[df['Borough'].str.contains("(?i)toronto" )]
df.shape

(39, 5)

### Using FourSquare API to explore neighbour


In [11]:
CLIENT_ID = 'DMGPTKDJFZ5YIWWLV51O5TPGH3UJQJX0B5XCGGAMAEK1EF4O' # your Foursquare ID
CLIENT_SECRET = 'K3JFSEMK40EG0NRQFSEYY41S5IL1VBO5NLT4NSPPEUJAP5BL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DMGPTKDJFZ5YIWWLV51O5TPGH3UJQJX0B5XCGGAMAEK1EF4O
CLIENT_SECRET:K3JFSEMK40EG0NRQFSEYY41S5IL1VBO5NLT4NSPPEUJAP5BL


#### Get 100 venues close to 500 m radius


In [14]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['PostalCode'], df['Borough'], df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

        
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighbourhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()   
    

(1638, 9)


,PostalCode,Borough,Neighbourhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = venues_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.0000,0.00,0.015625
5,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
6,Church and Wellesley,0.013158,0.000000,0.000000,0.000000,0.0,0.000000,0.013158,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.026316
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.0000,0.00,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000


In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Bakery,Restaurant,Café,Beer Bar,Cocktail Bar,Cheese Shop,Farmers Market,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Coffee Shop,Breakfast Spot,Convenience Store,Burrito Place,Restaurant,Italian Restaurant,Bar,Grocery Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Bar,Airport Terminal
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Department Store,Indian Restaurant


In [20]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Dessert Shop,Shoe Store,Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,Arts & Crafts Store,Bar,Bank,Café,Sushi Restaurant,Beer Bar,Distribution Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Italian Restaurant,Café,Cosmetics Shop,Theater,Tea Room,Electronics Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Restaurant,Clothing Store,Cosmetics Shop,American Restaurant,Cocktail Bar,Farmers Market,Moroccan Restaurant,Hotel
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Asian Restaurant,Neighborhood,Trail,Pub,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner
